In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

df_3_years=pd.read_excel("#")
column_values = df_3_years['#'].unique().tolist()
data = [{'id': "", 'januar':"",'Febuary':"",'March':"",'April':"",'Mai':"",'June':"",'july':"",'August':"",'September':"",'October':"",'November':"",'December':"" }]
# Create DataFrame

df_rest_of_year = pd.DataFrame(data)

# Function to calculate the occurrence count of a specific 'otto_code' for each 'month_year'
def count_occurrences(code):
    occurrences = df_3_years[df_3_years['#'] == code].groupby('month_year').size().reset_index(name='occurrence')
    return occurrences

column_values = df_3_years['#'].unique().tolist()
say=0

# Create DataFrame
df = pd.DataFrame(data)
for value in column_values:
    # Prompt user for otto_code input
    input_code = value

    # Generate the desired DataFrame
    store_sales = pd.DataFrame()
    months = pd.date_range(start='2019-08', end='2024-01', freq='M').strftime('%Y-%m').tolist()
  


    for month in months[:]:
        count = df_3_years[(df_3_years['month_year'] == month) & (df_3_years['sku'] == input_code)].shape[0]
        store_sales= pd.concat([store_sales, pd.DataFrame({'date': [month],'sales': [count]})]) #Dataframe for just spesific ottocode id
    

    store_sales["date"]=pd.to_datetime(store_sales["date"])
    store_sales["date"]=store_sales["date"].dt.to_period("M")
    monthly_sales=store_sales.groupby("date").sum().reset_index()
    monthly_sales["date"]=monthly_sales["date"].dt.to_timestamp()

    #visualization
    """"
    plt.figure(figsize=(15,5))
    plt.plot(monthly_sales["date"],monthly_sales["sales"])
    plt.xlabel("Date")
    plt.ylabel("Sales")
    plt.title("Monthly Customer Sales")
    plt.show()
    """
    #lets show the sales diffrences between the months
    monthly_sales["sales_diff"]=monthly_sales["sales"].diff()
    monthly_sales=monthly_sales.dropna()
    monthly_sales.head(12)
    """"
    plt.figure(figsize=(15,5))
    plt.plot(monthly_sales["date"],monthly_sales["sales_diff"])
    plt.xlabel("Date")
    plt.ylabel("Sales")
    plt.title("Monthly change of sales")
    plt.show()
    """
    #train to predict the sales on next month/ dropp of sales and date
    supervised_data=monthly_sales.drop(["date","sales"],axis=1)
    supervised_data.head(12) # that equals 1 year

    #first 12 month as input and second 12 month summ as label
    for i in range(1,13):
        col_name="month_"+str(i)
        supervised_data[col_name]=supervised_data["sales_diff"].shift(i)
    supervised_data=supervised_data.dropna().reset_index(drop=True)
    supervised_data.head(10)

    #split data into train and test data preprocessing
    train_data=supervised_data[:-12]
    test_data=supervised_data[-12:]
    print("Train data shape",train_data.shape)
    print("Test data shape",test_data.shape)

    scaler=MinMaxScaler(feature_range=(-1,1))
    scaler.fit(train_data)
    train_data=scaler.transform(train_data)
    test_data=scaler.transform(test_data)

    x_train,y_train=train_data[:,1:],train_data[:,0:1]
    x_test,y_test=test_data[:,1:],test_data[:,0:1]
    y_train=y_train.ravel()
    y_test=y_test.ravel()
    print("x_train shape:",x_train.shape)
    print("y_train shape:",y_train.shape)
    print("x_test shape:",x_test.shape)
    print("y_test shape:",y_test.shape)

    #prediction dataframe to mereg predicted sales price of all tranied algo
    sales_dates=monthly_sales["date"][-12:].reset_index(drop=True)
    predict_df=pd.DataFrame(sales_dates)
    act_sales=monthly_sales["sales"][-13:].to_list()

    from sklearn.ensemble import RandomForestRegressor
    lr_model=RandomForestRegressor(
    n_estimators= 30,
    min_samples_split= 14,
    min_samples_leaf=1,
    max_features= 0.5,
    max_depth=3,
    random_state=42)

    lr_model.fit(x_train,y_train)
    lr_pre= lr_model.predict(x_test)
    lr_pre=lr_pre.reshape(-1,1)#created lr prediction
    lr_pre_test_set=np.concatenate([lr_pre,x_test],axis=1)#matrix with input features of test data and predicted output
    lr_pre_test_set=scaler.inverse_transform(lr_pre_test_set)

    result_list=[]
    for index in range(0,len(lr_pre_test_set)):
      result_list.append(lr_pre_test_set[index][0] + act_sales[index])
    lr_pre_series=pd.Series(result_list,name="Linear Prediction")
    predict_df=predict_df.merge(lr_pre_series,left_index=True,right_index=True)

    print("hallo",predict_df["Linear Prediction"])


    lr_mse = np.sqrt(mean_squared_error(predict_df["Linear Prediction"],monthly_sales["sales"][-12:]))
    lr_mae=mean_absolute_error(predict_df["Linear Prediction"],monthly_sales["sales"][-12:])
    lr_r2=r2_score(predict_df["Linear Prediction"],monthly_sales["sales"][-12:])
    print("Linear Regression MSE:",lr_mse)
    print("Linear Regression MAE:",lr_mae)
    print("Linear Regression R2:",lr_r2)

    #visualize the predictio to av-ctual sales
    #plt.figure(figsize=(15,5))
    #actual sales
    #plt.plot(monthly_sales["date"],monthly_sales["sales"])
    #predicted sales
    """"
    plt.plot(predict_df["date"],predict_df["Linear Prediction"])
    plt.title("Customer sales forecast using linear regression")
    plt.xlabel("Date")
    plt.ylabel("sales")
    plt.legend(["Actual sales","Predicted sales"])
    plt.show()
    """
    column_list = predict_df['Linear Prediction'].tolist()
    new_row = {'id': value, 'januar':column_list[0],'Febuary':column_list[1],'March':column_list[2],'April':column_list[3],'Mai':column_list[4],'June':column_list[5],'july':column_list[6],'August':column_list[7],'September':column_list[8],'October':column_list[9],'November':column_list[10],'December':column_list[11] }
    df_rest_of_year.loc[say]=new_row

    say=say+1
    print(say)
df_rest_of_year.to_csv("June_predictions_2023",index=False)
df_rest_of_year.to_excel("june_predictions_2023")